In [1]:
import glob
import h5py
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
img_list = glob.glob('/bd_byta6000i0/users/surgical_depth/SCARED_fixed/**/*.png', recursive=True)

In [3]:
len(img_list)

28857

In [4]:
with h5py.File('/bd_byta6000i0/users/surgical_depth/SCARED_fixed/scared.hdf5', 'w') as hdf:

    for i, sample in enumerate(tqdm(img_list, desc='Processing')):
        parts = sample.split(os.sep)
        dataset_name = parts[-5]  # 'dataset5'
        keyframe_name = parts[-4]  # 'keyframe4'


        filename_with_extension = parts[-1]
        filename, extension = os.path.splitext(filename_with_extension)
        real_number = str(int(filename)).zfill(6)
        gt_number = str(int(filename) - 1).zfill(6)


        group_name = os.path.join(dataset_name, keyframe_name, real_number)

        img = np.array(Image.open(sample).convert('RGB'))


        # 修改文件名和目录
        parts[-2] = "data/groundtruth"  # 将'data'替换为'grundtruth'


        new_filename = f"scene_points{gt_number}.npy"


        parts[-1] = new_filename  

        gt_path = os.sep.join(parts)
        gt = np.load(gt_path).reshape(1024,1280,1)


        if group_name not in hdf:
            # 如果组不存在，则创建它
            hdf.create_group(group_name)
        
        group = hdf[group_name]

        group.create_dataset(name='image', data=img)
        group.create_dataset(name='gt', data=gt)
    


Processing: 100%|██████████| 28857/28857 [47:27<00:00, 10.13it/s]  


In [9]:
with h5py.File('/bd_byta6000i0/users/surgical_depth/SCARED_fixed/scared.hdf5', 'r') as hdf:
    batch = hdf['/dataset5/keyframe4/000001']['image'][:]

In [10]:
batch.shape

(1024, 1280, 3)